In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('kbli.xlsx')

In [3]:
df

,kode,deskripsi
0,90002,usaha telivisi
1,90002,kegiatan penari
2,90002,kegiatan aktor
3,90002,kegiatan penata musik
4,90002,usaha pelukis
...,...,...
126,3226,pembenihan buaya
127,3226,pembenihan katak
128,3226,Pembenihan ikan lele
129,3226,Pembenihan ikan gurame


In [4]:
df.kode

0      90002
1      90002
2      90002
3      90002
4      90002
       ...  
126     3226
127     3226
128     3226
129     3226
130     3226
Name: kode, Length: 131, dtype: int64

In [13]:
df

,kode,deskripsi
0,90002,usaha telivisi
1,90002,kegiatan penari
2,90002,kegiatan aktor
3,90002,kegiatan penata musik
4,90002,usaha pelukis
...,...,...
126,3226,pembenihan buaya
127,3226,pembenihan katak
128,3226,Pembenihan ikan lele
129,3226,Pembenihan ikan gurame


In [5]:
df['kode'].unique()

array([90002, 49424, 71101, 93192, 56290,  1270, 10130, 10110, 11040,
       10772, 61200,  3225,  3226], dtype=int64)

In [15]:
df['deskripsi'] = df['deskripsi'].astype('str')

### Stemming

In [6]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [7]:
from tqdm import tqdm

for i in tqdm(df.index):
  df.loc[i, 'deskripsi'] = stemmer.stem(df.loc[i, 'deskripsi'])

100%|████████████████████████████████████████████████████████████████████████████████| 131/131 [00:05<00:00, 24.74it/s]


In [8]:
df

,kode,deskripsi
0,90002,usaha telivisi
1,90002,giat tari
2,90002,giat aktor
3,90002,giat tata musik
4,90002,usaha peluk
...,...,...
126,3226,benih buaya
127,3226,benih katak
128,3226,benih ikan lele
129,3226,benih ikan gurame


### Stop Word Removal

In [9]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory2 = StopWordRemoverFactory()
stopword = factory2.create_stop_word_remover()
 
# Kalimat
for i in tqdm(df.index):
  df.loc[i, 'deskripsi'] = stopword.remove(df.loc[i, 'deskripsi'])

100%|██████████████████████████████████████████████████████████████████████████████| 131/131 [00:00<00:00, 1622.45it/s]


In [10]:
df.head()

,kode,deskripsi
0,90002,usaha telivisi
1,90002,giat tari
2,90002,giat aktor
3,90002,giat tata musik
4,90002,usaha peluk


In [11]:
df['kode'] = df['kode'].astype('str')

In [12]:
df['kode'].unique()

array(['90002', '49424', '71101', '93192', '56290', '1270', '10130',
       '10110', '11040', '10772', '61200', '3225', '3226'], dtype=object)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.deskripsi, df.kode, test_size=0.20, random_state=5, stratify=df.kode)

In [ ]:
datatrain = pd.DataFrame()

In [22]:
Train = pd.DataFrame({
    'A': X_train,
    'B': y_train
})

In [23]:
Train

,A,B
81,usaha buat air soda,11040
26,usaha konsultasi arsitektur inspeksi,71101
14,tukang ojek,49424
12,gojek,49424
27,jasa arsitektur,71101
...,...,...
63,usaha awet daging,10130
46,makan dapur asrama sekolah,56290
96,usaha buat bubuk kayu manis,10772
65,olah daging sapi beku potong,10130


In [21]:
Train

81                     usaha buat air soda
26    usaha konsultasi arsitektur inspeksi
14                             tukang ojek
12                                   gojek
27                         jasa arsitektur
                      ...                 
63                                   10130
46                                   56290
96                                   10772
65                                   10130
88                                   11040
Length: 208, dtype: object

In [17]:
mat = (X_train,y_train)

In [18]:
mat


(81                     usaha buat air soda
 26    usaha konsultasi arsitektur inspeksi
 14                             tukang ojek
 12                                   gojek
 27                         jasa arsitektur
                       ...                 
 63                       usaha awet daging
 46              makan dapur asrama sekolah
 96             usaha buat bubuk kayu manis
 65            olah daging sapi beku potong
 88              usaha buat bir non alkohol
 Name: deskripsi, Length: 104, dtype: object, 81    11040
 26    71101
 14    49424
 12    49424
 27    71101
       ...  
 63    10130
 46    56290
 96    10772
 65    10130
 88    11040
 Name: kode, Length: 104, dtype: object)

In [14]:
y_test

106    61200
114     3225
40     56290
9      90002
21     71101
66     10130
97     10772
84     11040
118     3225
110    61200
34     93192
55      1270
16     49424
31     93192
57      1270
127     3226
121     3226
28     71101
2      90002
91     10772
77     10110
92     10772
64     10130
71     10110
83     11040
18     49424
42     56290
Name: kode, dtype: object

### Model

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8888888888888888
              precision    recall  f1-score   support

       10110       0.67      1.00      0.80         2
       10130       1.00      1.00      1.00         2
       10772       1.00      1.00      1.00         3
       11040       1.00      1.00      1.00         2
        1270       1.00      1.00      1.00         2
        3225       1.00      0.50      0.67         2
        3226       0.67      1.00      0.80         2
       49424       1.00      1.00      1.00         2
       56290       1.00      1.00      1.00         2
       61200       1.00      1.00      1.00         2
       71101       1.00      1.00      1.00         2
       90002       0.00      0.00      0.00         2
       93192       0.67      1.00      0.80         2

    accuracy                           0.89        27
   macro avg       0.85      0.88      0.85        27
weighted avg       0.85      0.89      0.86        27



c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
from sklearn.model_selection import cross_val_score
clf = nb
scores = cross_val_score(clf, df.deskripsi, df.kode, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.80 (+/- 0.09)


In [30]:
tqdm.pandas(desc= "bar")

c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [57]:
import joblib
filename = 'modelNB.pkl'
joblib.dump(nb,filename)

['modelNB.pkl']

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])
lr.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = lr.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8518518518518519
              precision    recall  f1-score   support

       10110       0.67      1.00      0.80         2
       10130       1.00      1.00      1.00         2
       10772       1.00      1.00      1.00         3
       11040       1.00      1.00      1.00         2
        1270       0.67      1.00      0.80         2
        3225       0.00      0.00      0.00         2
        3226       0.67      1.00      0.80         2
       49424       1.00      1.00      1.00         2
       56290       1.00      1.00      1.00         2
       61200       1.00      1.00      1.00         2
       71101       1.00      1.00      1.00         2
       90002       0.00      0.00      0.00         2
       93192       0.67      1.00      0.80         2

    accuracy                           0.85        27
   macro avg       0.74      0.85      0.78        27
weighted avg       0.75      0.85      0.79        27



c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
clf = lr
scores = cross_val_score(clf, df.deskripsi, df.kode, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.81 (+/- 0.04)


c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", F

In [34]:
clf = rf
scores = cross_val_score(clf, df.deskripsi, df.kode, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.68 (+/- 0.11)


In [58]:
import joblib
filename = 'modelLR.pkl'
joblib.dump(lr,filename)

['modelLR.pkl']

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

rf = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier()),
              ])
rf.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = rf.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.6296296296296297
              precision    recall  f1-score   support

       10110       0.67      1.00      0.80         2
       10130       1.00      1.00      1.00         2
       10772       0.50      0.33      0.40         3
       11040       0.33      0.50      0.40         2
        1270       0.40      1.00      0.57         2
        3225       0.00      0.00      0.00         2
        3226       0.50      0.50      0.50         2
       49424       1.00      1.00      1.00         2
       56290       1.00      1.00      1.00         2
       61200       1.00      0.50      0.67         2
       71101       1.00      0.50      0.67         2
       90002       0.00      0.00      0.00         2
       93192       0.67      1.00      0.80         2

    accuracy                           0.63        27
   macro avg       0.62      0.64      0.60        27
weighted avg       0.62      0.63      0.59        27



c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [35]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

svc = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier()),
              ])
svc.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = svc.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.7037037037037037
              precision    recall  f1-score   support

       10110       0.00      0.00      0.00         2
       10130       1.00      1.00      1.00         2
       10772       1.00      0.33      0.50         3
       11040       0.50      1.00      0.67         2
        1270       0.67      1.00      0.80         2
        3225       0.00      0.00      0.00         2
        3226       0.67      1.00      0.80         2
       49424       0.67      1.00      0.80         2
       56290       1.00      1.00      1.00         2
       61200       1.00      0.50      0.67         2
       71101       1.00      1.00      1.00         2
       90002       0.33      0.50      0.40         2
       93192       0.67      1.00      0.80         2

    accuracy                           0.70        27
   macro avg       0.65      0.72      0.65        27
weighted avg       0.67      0.70      0.64        27



c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [36]:
clf = svc
scores = cross_val_score(clf, df.deskripsi, df.kode, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.73 (+/- 0.18)


In [55]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

gb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier()),
              ])
gb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = gb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.7777777777777778
              precision    recall  f1-score   support

       10110       0.67      1.00      0.80         2
       10130       1.00      1.00      1.00         2
       10772       1.00      0.33      0.50         3
       11040       0.50      1.00      0.67         2
        1270       0.67      1.00      0.80         2
        3225       1.00      0.50      0.67         2
        3226       0.67      1.00      0.80         2
       49424       1.00      1.00      1.00         2
       56290       1.00      1.00      1.00         2
       61200       1.00      0.50      0.67         2
       71101       1.00      1.00      1.00         2
       90002       0.00      0.00      0.00         2
       93192       1.00      1.00      1.00         2

    accuracy                           0.78        27
   macro avg       0.81      0.79      0.76        27
weighted avg       0.81      0.78      0.75        27



c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [56]:
clf = gb
scores = cross_val_score(gb, df.deskripsi, df.kode, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.72 (+/- 0.24)


In [59]:
import joblib
filename = 'modelRF.pkl'
joblib.dump(rf,filename)

['modelRF.pkl']

In [51]:
nb.predict(['abang gojek'])

array(['49424'], dtype='<U5')

In [76]:

loaded_model = joblib.load('modelNB.pkl')
result = loaded_model.predict(['abang gojek'])
result[0].astype(int)

49424

In [75]:
result[0].astype(int)

49424